In [1]:
!pip install roboflow

^C


  Using cached roboflow-1.1.23-py3-none-any.whl.metadata (9.3 kB)
  Using cached certifi-2023.7.22-py3-none-any.whl.metadata (2.2 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl.metadata (722 bytes)
  Using cached idna-2.10-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached opencv_python_headless-4.8.0.74-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl.metadata (5.8 kB)
Using cached roboflow-1.1.23-py3-none-any.whl (71 kB)
Using cached certifi-2023.7.22-py3-none-any.whl (158 kB)
Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
   ---------------------------------------- 0.0/38.0 MB ? eta -:--:--
   -----

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="Q85BjKM3lXpRS3UGlGEg")
project = rf.workspace("university-e8im3").project("instance-segmentation-trash-plastic")
version = project.version(1)
dataset = version.download("yolov5-obb")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Instance-Segmentation-Trash-Plastic-1 in yolov5-obb:: 100%|██████████| 5012/5012 [00:01<00:00, 2729.07it/s]


In [3]:
!pip install torch torchvision torchaudio
!pip install opencv-python-headless

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
    --------------------------------------- 0.0/2.4 MB 325.1 kB/s eta 0:00:08
    --------------------------------------- 0.0/2.4 MB 279.3 kB/s eta 0:00:09
   - -------------------------------------- 0.1/2.4 MB 363.1 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.4 MB 435.7 kB/s eta 0:00:06
   -- ------------------------------------- 0.1/2.4 MB 532.5 kB/s eta 0:00:05
   -- ------------------------------------- 0.2/2.4 MB 551.6 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/2.4 MB 588.9 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/2.4 MB 518.8 kB/s eta 0:00:05
   ---- ----------------------------------- 0.3/2.4 MB 561.1 kB/s eta 0:00:04
   ---- ----------------------------------- 0.3/2.4 MB 567.7 kB/s eta 0:00:04
   ---- ----------------------------------- 0.3/2.4 MB 553.0 kB/s eta 0:00:04
   -

In [ ]:
import cv2
import torch

# Load the model
model = torch.hub.load('ultralytics/yolov5', 'custom',
                       path='path/to/your/model/best.pt')  # Update the path

# Initialize the webcam
cap = cv2.VideoCapture(0)  # 0 is usually the default webcam

if not cap.isOpened():
    print("Error: Could not open video device.")
else:
    # Set properties. Each property is represented by a number, 3 and 4 represent width and height.
    cap.set(3, 640)  # Set the width
    cap.set(4, 480)  # Set the height

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Convert frame to RGB (YOLOv5 expects RGB)
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Make predictions
    results = model([img], size=640)  # Adjust size if needed

    # Update frame with detections
    # xyxy are the bounding box coordinates
    for *xyxy, conf, cls in results.xyxy[0]:
        # You can customize this part to add labels, bounding box or confidence if needed.
        label = f'{results.names[int(cls)]} {conf:.2f}'
        cv2.rectangle(frame, (int(xyxy[0]), int(xyxy[1])), (int(
            xyxy[2]), int(xyxy[3])), (255, 0, 0), 2)
        cv2.putText(frame, label, (int(xyxy[0]), int(
            xyxy[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Display the resulting frame
    cv2.imshow('frame', frame)

    # Break the loop with 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

!python train.py --img 640 --batch 16 --epochs 50 --data dataset.yaml --weights yolov5s.pt --cache